## Predicting Energy Consumption from PJM (in MW)

Will also be comparing energy consumption from the East and West Regions of the United States as well as their projected energy consumption.

In [38]:
import pandas as pd
import seaborn as sbn
import plotly as plt
import numericalmethods as nm
import scipy.optimize as sp
import math
import numpy as np

In [25]:
# Importing the csv files and assigning them as dataframes
dfw = pd.read_csv('PJMW_hourly.csv')
dfe = pd.read_csv('PJME_hourly.csv')

West-region

In [37]:
# Using numerical methods to create a model from each dataset
pd.to_numeric(dfw['PJMW_MW'])

0         2002-12-31 01:00:00
1         2002-12-31 02:00:00
2         2002-12-31 03:00:00
3         2002-12-31 04:00:00
4         2002-12-31 05:00:00
                 ...         
143201    2018-01-01 20:00:00
143202    2018-01-01 21:00:00
143203    2018-01-01 22:00:00
143204    2018-01-01 23:00:00
143205    2018-01-02 00:00:00
Name: Datetime, Length: 143206, dtype: object

East-region

In [ ]:
# Using numerical methods to create a model from each dataset
